In [1]:
hostname="3.235.3.53"
username="ubuntu"
key_filename="test.pem"

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from kick import kick

In [3]:
# define a transformation pipeline for our raw data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# download raw data and then apply transformation pipeline
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# then construct a generator from the transformed data
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [4]:
class LittleBlock(nn.Module):
    def __init__(self, in_sz, out_sz):
        super(LittleBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_sz, out_channels=out_sz, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2)

    def forward(self, x):
        x = self.conv(x)
        x = F.relu(x)
        x = self.pool(x)
        return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()  # this is how objects that inherit from parent classes are instantiated
        
        self.unit1 = LittleBlock(in_sz=3, out_sz=6)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(in_features=16 * 8 * 8, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)  # lets use our custom layer
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        # think of these as the edges; they are transformations and are therefore stateless
        
        x = self.unit1(x)  # input is 3x32x32 and becomes 6x16x16        
        x = self.conv2(x)  # after 2nd convolution, input becomes 16x16x16
        x = F.relu(x)  
        x = self.pool(x)  # after pooling, input is 16x8x8
        
        x = x.view(-1, 16 * 8 * 8)  # flattens into 1d vector
        
        x = self.fc1(x)  # input becomes 120 dim
        x = F.relu(x)
        x = self.fc2(x)  # input goes from 120 to 84 dim
        x = F.relu(x)
        x = self.fc3(x)  # outputs raw logits rather than class probabilities
        return x

In [5]:
net = Net()

In [6]:
@kick
def results():
    dataiter = iter(trainloader)
    images, labels = dataiter.next()
    return net(images)

initialize


In [7]:
results()

b'Collecting torch.nn'
b'Collecting torch.optim'
b'Collecting torch.nn.functional'
b'Collecting torchvision.transforms'
b'Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz'
b'Extracting ./data/cifar-10-python.tar.gz to ./data'
b'tensor([[-0.0866, -0.0434, -0.0972, -0.0333, -0.0271,  0.0340, -0.0222,  0.0882,'
b'          0.0927,  0.0780],'
b'        [-0.0864, -0.0381, -0.0945, -0.0422, -0.0264,  0.0311, -0.0232,  0.0911,'
b'          0.0953,  0.0713],'
b'        [-0.0894, -0.0416, -0.0971, -0.0393, -0.0282,  0.0348, -0.0161,  0.0898,'
b'          0.0990,  0.0742],'
b'        [-0.0881, -0.0334, -0.0981, -0.0405, -0.0346,  0.0371, -0.0261,  0.0899,'
b'          0.0918,  0.0733],'
b'        [-0.0829, -0.0354, -0.0988, -0.0314, -0.0244,  0.0306, -0.0237,  0.0858,'
b'          0.0946,  0.0781],'
b'        [-0.0850, -0.0386, -0.0976, -0.0326, -0.0266,  0.0294, -0.0269,  0.0918,'
b'          0.0967,  0.0782],'
b'        [-0.0841, -0.0353, -0.